<a href="https://colab.research.google.com/github/vuongdenchoi/introductionwithR/blob/main/DSR_SQL_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
install.packages('RSQLite')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘plogr’




In [ ]:
# Load required libraries
library(DBI)
library(RSQLite)

# Database Creation and Basic Operations

Task 1: Create a database with two tables (departments and employees) and insert sample data.
* departments infomation and sample data

| dept_id	(PRIMARY KEY)| dept_name|	location|
| --- |	--- |	--- |
| \<int>| \<chr> | \<char> |
| 1 |	IT |	New York|
| 2 |	HR |	Chicago|
| 3 |	Sales |	Los Angeles|
| 4	| Marketing	|Boston|

* employees information and sample data

| emp_id (PRIMARY KEY) |	first_name |	last_name|	dept_id (FOREIGN KEY)|	salary|	hire_date|
|---|---|---|---|---|---|
|\<int>|	\<chr>	|\<chr>	|\<int>	|\<int>	|\<chr>|
|1|	John	|Smith|	1|	78750	|2020-01-15|
|2|	Mary	|Johnson|	2|	65000	|2019-03-20|
|3|	Robert	|Brown|	1|	84000	|2018-11-10|
|4|	Patricia|	Wilson|	3|	70000|	2021-02-28|
|5|	Michael	|Davis|	3|	72000|	2020-07-15|
|6|	Linda	|Garcia|	2|	67000	|2019-09-01|


In [ ]:
# Create connection to SQLite database
con <- dbConnect(RSQLite::SQLite(), "company.db")

# Create sample tables
dbExecute(con, "
  CREATE TABLE IF NOT EXISTS departments (
   dept_id INTEGER PRIMARY KEY,
   dept_name TEXT,
   location TEXT
  )")

dbExecute(con, "
  CREATE TABLE IF NOT EXISTS employees (
    emp_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    dept_id INTEGER,
    salary INTERGER,
    hire_date TEXT,
    FOREIGN KEY (dept_id) REFERENCES departments(dept_id)
  )")

# Insert sample data
dbExecute(con, "
  INSERT INTO departments (dept_id, dept_name, location) VALUES
  (1, 'IT', 'New York'),
  (2, 'HR', 'Chicago'),
  (3, 'Sales', 'Los Angeles'),
  (4, 'Marketing', 'Boston')")

dbExecute(con, "
  INSERT INTO employees (emp_id, first_name, last_name, dept_id, salary, hire_date) VALUES
  (1, 'John', 'Smith', 1, 78750, '2020-01-15'),
  (2, 'Mary', 'Johnson', 2, 65000, '2019-03-20'),
  (3, 'Robert', 'Brown', 1, 84000, '2018-11-10'),
  (4, 'Patricia', 'Wilson', 3, 70000, '2021-02-28'),
  (5, 'Michael', 'Davis', 3, 72000, '2020-07-15'),
  (6, 'Linda', 'Garcia', 2, 67000, '2019-09-01')")


[1] 0

[1] 0

[1] 4

[1] 6

Task 2:

2.1 Find employees with salary greater than 70000 and sort by salary in descending order.

2.2 Find employees working in New York location

2.3 Calculate department statistics for departments with more than 1 employee

In [ ]:
# Function to execute queries and display results

run_query <- function(query) {
  result <- dbGetQuery(con, query)
  print(result)
  return(result)
}

# 2.1
query1 <- "SELECT * FROM employees WHERE salary > 70000 ORDER BY salary DESC"

# 2.2
query2 <- "SELECT e.first_name, e.last_name
           FROM employees e
           JOIN departments d ON e.dept_id = d.dept_id
           WHERE d.location = 'New York'"

# 2.3
query3 <- "SELECT d.dept_name, COUNT(*) AS num_employees, AVG(e.salary) AS avg_salary
           FROM employees e
           JOIN departments d ON e.dept_id = d.dept_id
           GROUP BY d.dept_name
           HAVING COUNT(*) > 1"

run_query(query1)
run_query(query2)
run_query(query3)

  emp_id first_name last_name dept_id salary  hire_date
1      3     Robert     Brown       1  84000 2018-11-10
2      1       John     Smith       1  78750 2020-01-15
3      5    Michael     Davis       3  72000 2020-07-15


emp_id,first_name,last_name,dept_id,salary,hire_date
<int>,<chr>,<chr>,<int>,<int>,<chr>
3,Robert,Brown,1,84000,2018-11-10
1,John,Smith,1,78750,2020-01-15
5,Michael,Davis,3,72000,2020-07-15


  first_name last_name
1       John     Smith
2     Robert     Brown


first_name,last_name
<chr>,<chr>
John,Smith
Robert,Brown


  dept_name num_employees avg_salary
1        HR             2      66000
2        IT             2      81375
3     Sales             2      71000


dept_name,num_employees,avg_salary
<chr>,<int>,<dbl>
HR,2,66000
IT,2,81375
Sales,2,71000


Task 3: Write a funtion with input department name, calculate as below:

* Employee count
* Average salary
* Minimum salary
* Maximum salary
* Total payroll
* Average years employed

In [ ]:
# Task 3

analyze_department_stats <- function(dept_name) {
  query <- paste0("
    SELECT
      COUNT(*) AS EmployeeCount,                -- Số lượng nhân viên
      AVG(e.salary) AS AverageSalary,          -- Lương trung bình
      MIN(e.salary) AS MinimumSalary,          -- Lương tối thiểu
      MAX(e.salary) AS MaximumSalary,          -- Lương tối đa
      SUM(e.salary) AS TotalPayroll,           -- Tổng lương
      AVG(STRFTIME('%J', 'now') - STRFTIME('%J', e.hire_date)) / 365.0 AS AverageYearsEmployed  -- Thời gian làm việc trung bình (năm)
    FROM employees e
    JOIN departments d ON e.dept_id = d.dept_id
    WHERE d.dept_name = '", dept_name, "'")
  result <- dbGetQuery(con, query)
  return(result)
}

In [ ]:
# Test the functions
print("Department Statistics for IT:")
print(analyze_department_stats("IT"))

[1] "Department Statistics for IT:"
  EmployeeCount AverageSalary MinimumSalary MaximumSalary TotalPayroll
1             2         81375         78750         84000       162750
  AverageYearsEmployed
1             5.678247


Task 4: Write a function that gives a percentage raise to all employees in a specific department.

In [ ]:
# Task 4:

give_department_raise <- function(dept_id, raise_percentage) {
    query <- sprintf("
    UPDATE employees
    SET salary = salary * (1 + %f/100)
    WHERE dept_id = ",raise_percentage, dept_id)
    dbExecute(con, query)


 }

In [ ]:
# Test the functions
print("\nGiving 7% raise to IT department (dept_id = 1):")
give_department_raise(1, 7)



[1] "\nGiving 7% raise to IT department (dept_id = 1):"


Warning message in sprintf("\n    UPDATE employees \n    SET salary = salary * (1 + %f/100) \n    WHERE dept_id = ", :
“one argument not used by format '
    UPDATE employees 
    SET salary = salary * (1 + %f/100) 
    WHERE dept_id = '”


ERROR: Error: incomplete input


In [ ]:

query <- "SELECT * FROM employees;"
employees_data <- dbGetQuery(con, query)

print(employees_data)

  emp_id first_name last_name dept_id salary  hire_date
1      1       John     Smith       1  78750 2020-01-15
2      2       Mary   Johnson       2  65000 2019-03-20
3      3     Robert     Brown       1  84000 2018-11-10
4      4   Patricia    Wilson       3  70000 2021-02-28
5      5    Michael     Davis       3  72000 2020-07-15
6      6      Linda    Garcia       2  67000 2019-09-01


Task 5: Write a function, a summary department report showing:

* Department name
* Employee count
* Average salary
* Total payroll
* Earliest hire date
* Latest hire date
* Cost per employee = Total payroll / Employee count

In [ ]:
# Task 5

generate_department_report <- function() {
  report_query <- "
    SELECT
      d.dept_name AS DepartmentName,            -- Tên phòng ban
      COUNT(e.emp_id) AS EmployeeCount,         -- Số lượng nhân viên
      AVG(e.salary) AS AverageSalary,           -- Lương trung bình
      SUM(e.salary) AS TotalPayroll,            -- Tổng lương
      MIN(e.hire_date) AS EarliestHireDate,     -- Ngày tuyển dụng sớm nhất
      MAX(e.hire_date) AS LatestHireDate,       -- Ngày tuyển dụng gần nhất
      SUM(e.salary) / COUNT(e.emp_id) AS CostPerEmployee  -- Chi phí cho mỗi nhân viên
    FROM employees e
    JOIN departments d ON e.dept_id = d.dept_id
    GROUP BY d.dept_name                            -- Nhóm theo tên phòng ban
    ORDER BY d.dept_name"

  result <- dbGetQuery(con, report_query)
  return(result)
}


In [ ]:
# Test the functions
print("\nSummary Department Report:")
print(generate_department_report())


[1] "\nSummary Department Report:"
  DepartmentName EmployeeCount AverageSalary TotalPayroll EarliestHireDate
1             HR             2         66000       132000       2019-03-20
2             IT             2         81375       162750       2018-11-10
3          Sales             2         71000       142000       2020-07-15
  LatestHireDate CostPerEmployee
1     2019-09-01           66000
2     2020-01-15           81375
3     2021-02-28           71000


In [ ]:
# Clean up
dbDisconnect(con)